In [1]:
## Install packages
!pip install torch_geometric
!pip install torch_sparse
!pip install torch_scatter

     |████████████████████████████████| 308 kB 4.0 MB/s 
     |████████████████████████████████| 376 kB 41.1 MB/s 
     |████████████████████████████████| 45 kB 3.2 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.0-py3-none-any.whl size=513273 sha256=214e73ac591b3508796c41fc51a2cfc77eb40620d7b860b0c8a61b4b3f2fecd4
  Stored in directory: /root/.cache/pip/wheels/b5/22/3f/10c97635d8d73e012582116b4e449d47fc64b7246cc71085fc
Successfully built torch-geometric
     |████████████████████████████████| 43 kB 749 kB/s 
  Created wheel for torch-sparse: filename=torch_sparse-0.6.12-cp37-cp37m-linux_x86_64.whl size=510307 sha256=7c2de95a9028b1be967959c2dfe549c17b3597928c7e227a3bffae7d58adeab5
  Stored in directory: /root/.cache/pip/wheels/fb/e2/2f/44956c61e3299573ffe12da9d1374c7576ca0c5fb1fe1ed38c
Successfully built torch-sparse
  Created wheel for torch-scatter: filename=torch_scatter-2.0.8-cp37-cp37m-linux_x86_64.whl size=281098 sha256=722acc9f1e7241762c8470441685d37c86907

In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import f1_score, classification_report
from torch.nn import functional as F
from torch.nn import Sequential as Seq, Linear, ReLU
from torch import Tensor
from torch.utils.dlpack import to_dlpack, from_dlpack

from torch import optim
import torch.utils.data
from sklearn.model_selection import GridSearchCV
from torch.utils.data import TensorDataset, DataLoader

from scipy.spatial.distance import cdist

import sys
sys.path.append('../')

# # import multiwoz.conv_graph
# from multiwoz.conv_graph import MultiWozConvGraph
from GNN_untils_test import evaluate_model, LSTM_model, Perceptron, BiLSTM, RNN_model
from GNN_untils_test import SoftBCEWithLogitsLoss, validate_model, f1
from GNN_untils_test import CreateDatalist, GAT_emb3, SageModel3, MLPClassifier, validate_model_GNN, evaluate_model_GNN


# from conv_graph_gnn import MultiWozConvGraph_GNN
from conv_graph_selfplay_gnn import SelfPlayConvGraph
# from utils_and_torch import evaluate_model, Classifier
# from utils_and_torch import SoftBCEWithLogitsLoss, validate_model, f1

import networkx as nx
import torch_geometric.data
from torch_geometric.data import InMemoryDataset, Data, Batch
from torch_geometric.data import DataLoader
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, degree
from torch_geometric.nn import GraphConv, TopKPooling, GatedGraphConv, SAGEConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch_geometric.transforms as T
from torch_geometric.nn import GATConv

In [3]:
## Generate Graphs
net_depth = 2
train_graph_GNN = SelfPlayConvGraph(dir_name="./", file_names=['/train.json'], seq_length = 4, net_depth = net_depth)
dev_graph_GNN = SelfPlayConvGraph(dir_name="./", file_names=['/dev.json'], seq_length = 4, net_depth = net_depth)
test_graph_GNN = SelfPlayConvGraph(dir_name="./", file_names=['/test.json'], seq_length = 4, net_depth = net_depth)
eval_graph_GNN = SelfPlayConvGraph(dir_name="./", file_names=['/train.json', '/dev.json', '/test.json'], seq_length = 4, net_depth = net_depth)

-----------------------------------------------
Stats for ConvGraph for .//train.json
Average degree: 2.013 (excluding outliers)
Number of nodes: 472
Number of edges: 1002
Number of conversations: 384
Unique turns: 978
Total turns: 3562
As a percentage: 27.456
-----------------------------------------------
-----------------------------------------------
Stats for ConvGraph for .//dev.json
Average degree: 1.857 (excluding outliers)
Number of nodes: 279
Number of edges: 518
Number of conversations: 120
Unique turns: 498
Total turns: 1134
As a percentage: 43.915
-----------------------------------------------
-----------------------------------------------
Stats for ConvGraph for .//test.json
Average degree: 1.820 (excluding outliers)
Number of nodes: 436
Number of edges: 857
Number of conversations: 264
Unique turns: 842
Total turns: 2464
As a percentage: 34.172
-----------------------------------------------
-----------------------------------------------
Stats for ConvGraph for .//tra

In [4]:
## GNN subgraphs data
x_train_GNN, y_train_GNN, edge_index_train, x_train_simple = train_graph_GNN.generate_subgraph_data()
x_dev_GNN, y_dev_GNN, edge_index_dev, x_dev_simple = dev_graph_GNN.generate_subgraph_data()
x_test_GNN, y_test_GNN, edge_index_test, x_test_simple = test_graph_GNN.generate_subgraph_data()


-----------------------------------------------
Stats for ConvGraph for .//train.json
Subgraph list created successfully.
Number of subgraphs created: 1589
-----------------------------------------------
-----------------------------------------------
Stats for ConvGraph for .//dev.json
Subgraph list created successfully.
Number of subgraphs created: 507
-----------------------------------------------
-----------------------------------------------
Stats for ConvGraph for .//test.json
Subgraph list created successfully.
Number of subgraphs created: 1100
-----------------------------------------------


In [5]:
print(len(x_train_GNN))
print(len(x_train_GNN[1][-1]))
print(len(y_train_GNN[1]))

## x of length 31
## y of length 12

1589
31
12


In [6]:
max_num_nodes_train = max([len(x_train_GNN[i]) for i in range(len(x_train_GNN))])
max_num_nodes_test = max([len(x_test_GNN[i]) for i in range(len(x_test_GNN))])
max_num_nodes_dev = max([len(x_dev_GNN[i]) for i in range(len(x_dev_GNN))])

print(max_num_nodes_train, max_num_nodes_test, max_num_nodes_dev)

min_num_nodes_train = min([len(x_train_GNN[i]) for i in range(len(x_train_GNN))])
min_num_nodes_test = min([len(x_test_GNN[i]) for i in range(len(x_test_GNN))])
min_num_nodes_dev = min([len(x_dev_GNN[i]) for i in range(len(x_dev_GNN))])

print(min_num_nodes_train, min_num_nodes_test, min_num_nodes_dev)


56 56 35
2 2 2


In [7]:
## Generate test data for simple search in search_graph
train_graph = train_graph_GNN.graph
# search_graph = train_dev_graph.graph
x_test = [x_test_GNN[i][-1] for i in range(len(x_test_GNN))]
# x_test_unique = np.unique(x_test, axis = 0)

x_dev = [x_dev_GNN[i][-1] for i in range(len(x_dev_GNN))]
# x_dev_unique = np.unique(x_dev, axis = 0)

x_seen_train = [x_train_simple[i][-1] for i in range(len(x_train_simple))]
x_seen_dev = [x_dev_GNN[i][-1] for i in range(len(x_dev_GNN))]
x_seen = np.concatenate((x_seen_train, x_seen_dev))

print('Simple search data created successfully.')

Simple search data created successfully.


### Create graph data
Each x_i in the dataset is a set of nodes for one individual graph. Every graph has different number of nodes N_i with shape [N_i, 31]. 

Each graph has one label y_i of shape [12]. The aim is to predict the label for a given graph.

In [8]:
def CreateDatalist(x_data, y_data, edge_data):
    data_list = []
        
    num_data = len(x_data)
        
    for i in range(num_data):
        x = torch.tensor(x_data[i])
        y = torch.tensor(y_data[i])
        edge_index = torch.tensor(edge_data[i]).t().contiguous()
            
        data = Data(x = x, y = y, edge_index = edge_index)
        data_list.append(data)
    return data_list

In [9]:
train_datalist = CreateDatalist(x_train_GNN, y_train_GNN, edge_index_train)
dev_datalist = CreateDatalist(x_dev_GNN, y_dev_GNN, edge_index_dev)
test_datalist = CreateDatalist(x_test_GNN, y_test_GNN, edge_index_test)

In [10]:
## Batch data droplast
batch_size = 32
train_loader = DataLoader(train_datalist, batch_size = batch_size,drop_last=True)
dev_loader = DataLoader(dev_datalist, batch_size = batch_size,drop_last=True)
test_loader = DataLoader(test_datalist, batch_size = batch_size,drop_last=True)

print("Batch created successfully.")

Batch created successfully.


/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [11]:
## Testrun batch
testrun_batch = next(iter(train_loader))

### Graph attention model

In [12]:
## Three layers of attention for three layers of neighbours
class GAT_emb(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GAT_emb, self).__init__()
        
        self.conv1 = GATConv(in_channels, 8, heads=1)
        self.conv2 = GATConv(8, 8, heads=1, concat=False)
        self.conv3 = GATConv(8, out_channels, heads=1, concat=False)
                

    def forward(self, data):
        x = data.x.float()
        edge_index = data.edge_index
        x = F.elu(self.conv1(x, edge_index))
        x = F.elu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        x = F.log_softmax(x, dim=-1)
        
        ## global mean pooling
        x = gmp(x, data.batch)  ## max pooling
        # x = gap(x, data.batch) ## mean pooling

        # x = x[-1]
        
        return x

In [13]:
## Test run
model = GAT_emb(31,31)
temp = model(testrun_batch)

print(temp.shape)

torch.Size([32, 31])


### GraphSage model

In [14]:
## Three layers of sage for three layers of neighbours
class SageModel(torch.nn.Module):
    def __init__(self, in_channels, dim_hidden_sage, out_channels):
        super(SageModel,self).__init__()
        self.conv1 = SAGEConv(in_channels, dim_hidden_sage)
        self.conv2 = SAGEConv(dim_hidden_sage, 32) 
        self.conv3 = SAGEConv(32, out_channels)
        
    def forward(self, data):
        x = data.x.float()
        edge_index = data.edge_index
        
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        x = F.log_softmax(x, dim=1)
        
        ## global mean pooling
        # x = gap(x, data.batch)
        x = gmp(x, data.batch)
        
        return x
        

In [15]:
## Test run
model = SageModel(31, 64, 31)
temp = model(testrun_batch)

print(temp.shape)

torch.Size([32, 31])


In [16]:
#@title
# class SAGEConv(MessagePassing):
#     def __init__(self, in_channels, out_channels):
#         super(SAGEConv, self).__init__(aggr='mean') #  "mean" aggregation.
#         self.lin = torch.nn.Linear(in_channels, out_channels)
#         self.act = torch.nn.ReLU()
#         self.update_lin = torch.nn.Linear(in_channels + out_channels, in_channels, bias=False)
#         self.update_act = torch.nn.ReLU()
        
        
#     def forward(self, data):
#         '''
#         x: [N, in_channels]
#         edge_index: [2, E]
#         x_j: [E, in_channels]
#         aggr_out: [N, out_channels]
        
#         '''  
#         x = data.x.float()
#         edge_index = data.edge_index
        
#         edge_index, _ = remove_self_loops(edge_index)
#         edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0)) 
        
#         return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

#     def message(self, x_j):
#         x_j = self.lin(x_j)
#         x_j = self.act(x_j)
        
#         return x_j

#     def update(self, aggr_out, x):
#         new_embedding = torch.cat([aggr_out, x], dim=1)
        
#         new_embedding = self.update_lin(new_embedding)
#         new_embedding = self.update_act(new_embedding)
        
#         return new_embedding

### Depth 2 GNN models

In [17]:
## Three layers of attention for three layers of neighbours
class GAT_emb(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GAT_emb, self).__init__()
        
        self.conv1 = GATConv(in_channels, 8, heads=1)
        self.conv2 = GATConv(8, out_channels, heads=1, concat=False)
                

    def forward(self, data):
        x = data.x.float()
        edge_index = data.edge_index
        x = F.elu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        x = F.log_softmax(x, dim=-1)
        
        ## global mean pooling
        # x = gap(x, data.batch)
        x = gmp(x, data.batch)

        return x

In [18]:
## Test run
model = GAT_emb(31,31)
temp = model(testrun_batch)

print(temp.shape)

torch.Size([32, 31])


In [19]:
## Three layers of sage for three layers of neighbours
class SageModel(torch.nn.Module):
    def __init__(self, in_channels, dim_hidden_sage, out_channels):
        super(SageModel,self).__init__()
        self.conv1 = SAGEConv(in_channels, dim_hidden_sage)
        self.conv2 = SAGEConv(dim_hidden_sage, out_channels)
        
    def forward(self, data):
        x = data.x.float()
        edge_index = data.edge_index
        
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = F.log_softmax(x, dim=1)
        
        ## global mean pooling
        x = gap(x, data.batch)
        
        return x
        

In [20]:
## Test run
model = SageModel(31, 64, 31)
temp = model(testrun_batch)

print(temp.shape)

torch.Size([32, 31])


### Depth 1 GNN models

In [21]:
## Three layers of attention for three layers of neighbours
class GAT_emb(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GAT_emb, self).__init__()
        
        self.conv1 = GATConv(in_channels, out_channels, heads=2, concat=False)
                

    def forward(self, data):
        x = data.x.float()
        edge_index = data.edge_index

        x = F.elu(self.conv1(x, edge_index))
        x = F.log_softmax(x, dim=-1)
        
        ## global mean pooling
        # x = gap(x, data.batch)
        x = gmp(x, data.batch)

        return x

In [22]:
## Test run
model = GAT_emb(31,31)
temp = model(testrun_batch)

print(temp.shape)

torch.Size([32, 31])


In [23]:
## Three layers of sage for three layers of neighbours
class SageModel(torch.nn.Module):
    def __init__(self, in_channels, dim_hidden_sage, out_channels):
        super(SageModel,self).__init__()
        self.conv1 = SAGEConv(in_channels, out_channels)
        
    def forward(self, data):
        x = data.x.float()
        edge_index = data.edge_index
        
        x = self.conv1(x, edge_index)
        x = F.log_softmax(x, dim=1)
        
        ## global mean pooling
        # x = gap(x, data.batch)
        x = gmp(x, data.batch)

        return x
        

In [24]:
## Test run
model = SageModel(31, 256, 31)
temp = model(testrun_batch)

print(temp.shape)

torch.Size([32, 31])


### MLP model

In [25]:
class MLPClassifier(torch.nn.Module):

    def __init__(self, emb_model, dim_input, dim_target, dim_hidden):
        super(MLPClassifier, self).__init__()
        '''
        dim_input: dimension of node feature (355)
        dim_target: dimension of target y (309)
        dim_hidden: number of hidden nodes
        max_num_nodes: max number of nodes in subgraphs (not used here)
        '''

        self.emb = emb_model
        self.fc_global = Linear(dim_input, dim_hidden)
        self.out = Linear(dim_hidden, dim_target)

    def forward(self, data):
        x = self.emb(data)
        x = self.out(F.relu(self.fc_global(x))) 

        return x


### Define training process

In [26]:
def validate_model_GNN(model, dataloader, convgraph, evel_graph, data_simple, train_with_soft_loss, valid_with_soft_F1):
    model.eval()
    if train_with_soft_loss:
        loss_function = SoftBCEWithLogitsLoss()
    else:
        loss_function = nn.BCEWithLogitsLoss()
        
    with torch.no_grad():
        valid_loss = 0
        valid_f1s = []
        valid_losses = []

        count = 1
        for batch in dataloader:
            output = model(batch)

            inputs = data_simple[(count-1)*batch_size:count*batch_size]
            labels = batch.y.reshape(batch_size, 12)

            count += 1

            if valid_with_soft_F1:
                for inp, out, lab in zip(inputs, output, labels):
                    y_pred = (out > 0.0).int()
                    nearest_gold, best_f1 = evel_graph.get_best_f1_score(inp.data.tolist(), y_pred.data.tolist())
                    valid_f1s.append(best_f1)
            else:
                for out, lab in zip(output, labels):
                    valid_f1s.append(f1((out > 0.0).int(), lab).cpu())

            if train_with_soft_loss:
                loss = loss_function(output.float(),batch.x.float(), convgraph)
            else:
                loss = loss_function(output.float(), labels.float())   
                
            valid_losses.append(loss.data.mean().cpu())
                
    model.train()
    return np.mean(valid_losses), np.mean(valid_f1s)

In [27]:
def evaluate_model_GNN(model, dataloader, data_simple, eval_graph, report=False):
    model.eval()
    with torch.no_grad():
        gold_labels = []
        nearest_gold_labels = []
        pred_labels = []
        soft_f1s = []
        
        count = 1
        for batch in dataloader:
            output = model(batch)
            # inputs = batch.x
            inputs = data_simple[(count-1)*batch_size:count*batch_size]
            labels = batch.y.reshape(batch_size, 12)

            count +=1

            for inp, out, lab in zip(inputs, output, labels):
                y_pred = (out > 0.0).int()
                pred_labels.append(y_pred.cpu().numpy())
                gold_labels.append(lab.cpu().numpy())
                # print(torch.tensor(inp).shape)
                nearest_gold, best_f1 = eval_graph.get_best_f1_score(inp.data.tolist(), y_pred.data.tolist())
                # nearest_gold, best_f1 = get_best_f1_score(inp.data.tolist(), y_pred.data.tolist())
                soft_f1s.append(best_f1)
                nearest_gold_labels.append(nearest_gold)

            
        print("Hard F-Score (exact match): %.3f" % f1_score(y_true=np.array(gold_labels, dtype='float32'), y_pred=np.array(pred_labels, dtype='float32'), average='samples'))
        print("Soft F-Score (best match): %f" % np.mean(soft_f1s))
        if report:
            print(classification_report(y_true=np.array(nearest_gold_labels, dtype='float32'),
                                        y_pred=np.array(pred_labels, dtype='float32'),
                                        target_names=eval_graph.dialog_act_to_idx.keys(), digits=3))
    model.train()

In [28]:
## Define F1 score
def f1(y_pred, y_true):
    if y_pred.ndim == 2:
        y_pred = y_pred.argmax(dim=1)

    tp = (y_true * y_pred).sum().to(torch.float32)
    fp = ((1 - y_true) * y_pred).sum().to(torch.float32)
    fn = (y_true * (1 - y_pred)).sum().to(torch.float32)

    epsilon = 1e-7
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    return 2 * (precision * recall) / (precision + recall + epsilon)

In [29]:
def load_checkpoint(model, optimizer, filename='checkpoint'):
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("=> loaded checkpoint '{}' (epoch {})".format(filename, checkpoint['epoch']))
    return model, optimizer

### Model training

In [30]:
## Model parameters
dim_input = torch.tensor(x_train_GNN[1]).shape[1]   ## 31
dim_target = torch.tensor(y_train_GNN[1]).shape[0]  ## 12

## sage train with max pooling
dim_hidden = 64
dim_hidden_sage = 64  ## hidden nodes for GraphSage

batch_size = 32


In [31]:
train_with_soft_loss = False

if train_with_soft_loss:
    loss_function = SoftBCEWithLogitsLoss()
else:
    loss_function = nn.BCEWithLogitsLoss()

In [32]:
## GAT training (example run for 5 epochs)
emb_model = GAT_emb(dim_input, dim_input)

model = MLPClassifier(emb_model, dim_input, dim_target, dim_hidden)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

max_epochs, max_val_f1, patience = 5, 0, 2

for epoch in range(max_epochs):
    f1s = []
    losses = []
    for batch in train_loader:
        optimizer.zero_grad()
        
        outputs = model(batch)
        labels = batch.y.reshape(batch_size, 12)
        inputs = batch.x
        
        if train_with_soft_loss:
            loss = loss_function(outputs.float(),inputs.float(), train_graph)
        else:
            loss = loss_function(outputs.float(), labels.float())

        loss.backward()
        optimizer.step()
        losses.append(loss.data.mean().cpu())

        for out, lab in zip(outputs, labels):
            f1s.append(f1((out > 0.0).float(), lab).cpu())


    valid_loss, valid_f1 = validate_model_GNN(model, dev_loader, None, eval_graph_GNN, x_dev_simple, train_with_soft_loss, False)
    # valid_loss, valid_f1 = validate_model_GNN_soft(model, dev_loader, x_dev_simple, None, train_with_soft_loss)
    # noinspection PyStringFormat
    print('[%d/%d] Train Loss: %.3f, Train F1: %.3f, Val Loss: %.3f, Val F1: %.3f,' % (epoch + 1, max_epochs, np.mean(losses), np.mean(f1s), valid_loss, valid_f1))

    # Early stopping
    if valid_f1 > max_val_f1:
        state = {'epoch': epoch + 1, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict(), }
        torch.save(state, 'checkpoint')

        max_val_f1 = valid_f1
        max_epochs = 0
    else:
        max_epochs += 1
        if max_epochs >= patience:
            model, optimizer = load_checkpoint(model, optimizer)
            print("Stopped early and went back to Validation f1: %.3f" % max_val_f1)
            break


[1/5] Train Loss: 0.497, Train F1: 0.339, Val Loss: 0.462, Val F1: 0.448,
[2/0] Train Loss: 0.442, Train F1: 0.430, Val Loss: 0.411, Val F1: 0.447,
[3/1] Train Loss: 0.399, Train F1: 0.489, Val Loss: 0.396, Val F1: 0.506,
[4/0] Train Loss: 0.375, Train F1: 0.540, Val Loss: 0.393, Val F1: 0.508,
[5/0] Train Loss: 0.363, Train F1: 0.562, Val Loss: 0.392, Val F1: 0.513,


In [33]:
### Results for an example run of only 5 epochs. For full training, run for more epochs.
print("---------------------- DEVELOPMENT SET REPORT --------------------------")
evaluate_model_GNN(model, dev_loader, x_dev_simple ,eval_graph_GNN)

print("--------------------- DEDUPLICATED TEST SET REPORT -------------------------")
evaluate_model_GNN(model, test_loader, x_test_simple, eval_graph_GNN)

---------------------- DEVELOPMENT SET REPORT --------------------------
Hard F-Score (exact match): 0.513
Soft F-Score (best match): 0.738276
--------------------- DEDUPLICATED TEST SET REPORT -------------------------
Hard F-Score (exact match): 0.543
Soft F-Score (best match): 0.729661


### GraphSAGE training

In [34]:
## Model parameters
dim_input = torch.tensor(x_train_GNN[1]).shape[1]   ## 31
dim_target = torch.tensor(y_train_GNN[1]).shape[0]  ## 12

## sage train with max pooling
dim_hidden = 128
dim_hidden_sage = 64  ## hidden nodes for GraphSage

batch_size = 32

In [35]:
train_with_soft_loss = False

if train_with_soft_loss:
    loss_function = SoftBCEWithLogitsLoss()
else:
    loss_function = nn.BCEWithLogitsLoss()

In [36]:
## GraphSage training (example run for 5 epochs only)
emb_model = SageModel(dim_input, dim_hidden_sage, dim_input)

model = MLPClassifier(emb_model, dim_input, dim_target, dim_hidden)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

max_epochs, max_val_f1, patience = 5, 0, 2
# max_epochs, max_val_f1, patience = 500, 0, 50

for epoch in range(max_epochs):
    f1s = []
    losses = []
    for batch in train_loader:
        optimizer.zero_grad()
        
        outputs = model(batch)
        labels = batch.y.reshape(batch_size, 12)
        inputs = batch.x
        
        if train_with_soft_loss:
            loss = loss_function(outputs.float(),inputs.float(), train_graph)
        else:
            loss = loss_function(outputs.float(), labels.float())

        loss.backward()
        optimizer.step()
        losses.append(loss.data.mean().cpu())

        for out, lab in zip(outputs, labels):
            f1s.append(f1((out > 0.0).float(), lab).cpu())


    valid_loss, valid_f1 = validate_model_GNN(model, dev_loader, None, eval_graph_GNN, x_dev_simple, train_with_soft_loss, False)
    # valid_loss, valid_f1 = validate_model_GNN_soft(model, dev_loader, x_dev_simple, None, train_with_soft_loss)
    # noinspection PyStringFormat
    print('[%d/%d] Train Loss: %.3f, Train F1: %.3f, Val Loss: %.3f, Val F1: %.3f,' % (epoch + 1, max_epochs, np.mean(losses), np.mean(f1s), valid_loss, valid_f1))

    # Early stopping
    if valid_f1 > max_val_f1:
        state = {'epoch': epoch + 1, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict(), }
        torch.save(state, 'checkpoint')

        max_val_f1 = valid_f1
        max_epochs = 0
    else:
        max_epochs += 1
        if max_epochs >= patience:
            model, optimizer = load_checkpoint(model, optimizer)
            print("Stopped early and went back to Validation f1: %.3f" % max_val_f1)
            break


[1/5] Train Loss: 0.504, Train F1: 0.291, Val Loss: 0.478, Val F1: 0.384,
[2/0] Train Loss: 0.474, Train F1: 0.377, Val Loss: 0.467, Val F1: 0.443,
[3/0] Train Loss: 0.453, Train F1: 0.435, Val Loss: 0.443, Val F1: 0.429,
[4/1] Train Loss: 0.423, Train F1: 0.474, Val Loss: 0.421, Val F1: 0.445,
[5/0] Train Loss: 0.403, Train F1: 0.512, Val Loss: 0.410, Val F1: 0.454,


In [37]:
### Results for an example run of only 5 epochs. For full training, run for more epochs.
print("---------------------- DEVELOPMENT SET REPORT --------------------------")
evaluate_model_GNN(model, dev_loader, x_dev_simple ,eval_graph_GNN)

print("--------------------- DEDUPLICATED TEST SET REPORT -------------------------")
evaluate_model_GNN(model, test_loader, x_test_simple, eval_graph_GNN)

---------------------- DEVELOPMENT SET REPORT --------------------------
Hard F-Score (exact match): 0.454
Soft F-Score (best match): 0.639206
--------------------- DEDUPLICATED TEST SET REPORT -------------------------
Hard F-Score (exact match): 0.463
Soft F-Score (best match): 0.623753


### Simple Search Model

In [38]:
train_with_soft_loss = False
if train_with_soft_loss:
    loss_function = SoftBCEWithLogitsLoss()
else:
    loss_function = nn.BCEWithLogitsLoss()

In [39]:
def find_nearest_node(node, node_seen, method):
    node = np.array(node).reshape(1,-1)
    dist = cdist(node, node_seen, method)

    idx = np.argmin(dist)
    nearest_node = node_seen[idx]

    return list(nearest_node)

In [40]:
def simple_search(dataset, data_seen, graph, method):
    y_pred = []
    ## Count how many nodes not seen in search graph
    count_oov = 0
    for data in dataset:
        node = str(list(data))
        if graph.has_node(node):
            dialog_acts = [(graph[node][t], t) for t in graph[node]]
            dialog_acts.sort(key=lambda t: t[0]['probability'], reverse=True)
            for dialog_act in dialog_acts[:1]:
                y = eval(dialog_act[1])[19:]
                y_pred.append(y)
        else:
            count_oov += 1

            data_hat = find_nearest_node(data, data_seen, method)
            data_hat = list(map(int, data_hat))
            node = str(data_hat)
            if graph.has_node(node) == False:
                print('incorrect')
            dialog_acts = [(graph[node][t], t) for t in graph[node]]
            dialog_acts.sort(key=lambda t: t[0]['probability'], reverse=True)
            for dialog_act in dialog_acts[:1]:
                y = eval(dialog_act[1])[19:]
                y_pred.append(y)
    return y_pred, count_oov


In [45]:
use_search = True
## method_list = {'euclidean', 'jaccard'}
method = 'jaccard'

if use_search == True:
    f1s_test = []
    ## Test set results
    outputs, test_oov = simple_search(x_test, x_seen_train, train_graph, method)

    outputs = torch.tensor(outputs).float()
    labels = torch.tensor(y_test_GNN).float()

    inputs = x_test_simple

    soft_f1s = []

    if train_with_soft_loss:
        loss_test = loss_function(outputs, x_test, train_graph_GNN)
    else:
        loss_test = loss_function(outputs, labels)

    for out, lab in zip(outputs, labels):
        f1s_test.append(f1((out > 0.0).float(), lab).cpu())

    for inp, out, lab in zip(inputs, outputs, labels):
        y_pred = (out > 0.0).int()
        nearest_gold, best_f1 = eval_graph_GNN.get_best_f1_score(inp.data.tolist(), y_pred.data.tolist())

        soft_f1s.append(best_f1)

    print('Testset result of Simple Search Method:', 'Test loss:', loss_test.data.mean().cpu(), 'Test F1:', np.mean(f1s_test))
    print('Number of test nodes not found in the search graph:', test_oov)
    print('Testset Soft F1: ', np.mean(soft_f1s))

    ##################################
    ## Dev set results
    f1s_dev = []
    soft_f1s_dev = []

    outputs_dev, dev_oov = simple_search(x_dev, x_seen_train, train_graph, method)

    outputs_dev = torch.tensor(outputs_dev).float()
    labels_dev = torch.tensor(y_dev_GNN).float()

    inputs_dev = x_dev_simple

    if train_with_soft_loss:
        loss_dev = loss_function(outputs_dev, x, train_graph_GNN)
    else:
        loss_dev = loss_function(outputs_dev, labels_dev)

    for out, lab in zip(outputs_dev, labels_dev):
        f1s_dev.append(f1((out > 0.0).float(), lab).cpu())

    for inp, out, lab in zip(inputs_dev, outputs_dev, labels_dev):
        y_pred = (out > 0.0).int()
        nearest_gold, best_f1 = eval_graph_GNN.get_best_f1_score(inp.data.tolist(), y_pred.data.tolist())

        soft_f1s_dev.append(best_f1)

    print('Devset result of Simple Search Method:', 'Valid loss:', loss_dev.data.mean().cpu(), 'Valid F1:', np.mean(f1s_dev))
    print('Number of dev nodes not found in the search graph:', dev_oov)
    print('Testset Soft F1: ', np.mean(soft_f1s_dev))

Testset result of Simple Search Method: Test loss: tensor(0.6759) Test F1: 0.6571501
Number of test nodes not found in the search graph: 90
Testset Soft F1:  0.9692056277056277
Devset result of Simple Search Method: Valid loss: tensor(0.6943) Valid F1: 0.60050243
Number of dev nodes not found in the search graph: 33
Testset Soft F1:  0.9752465483234715


### Sequential models

In [49]:
### Data loader for LinearData
x_train, y_train = train_graph_GNN.generate_standard_data(unique=False)
x_dev, y_dev = dev_graph_GNN.generate_standard_data(unique=False)
x_test, y_test = test_graph_GNN.generate_standard_data(unique=True)

params = {'batch_size': 32, 'shuffle': True}

training_set = TensorDataset(torch.tensor(x_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
training_generator = torch.utils.data.DataLoader(training_set, **params)
validation_set = TensorDataset(torch.tensor(x_dev, dtype=torch.float32), torch.tensor(y_dev, dtype=torch.float32))
validation_generator = torch.utils.data.DataLoader(validation_set, **params)
no_dupl_test_set = TensorDataset(torch.tensor(x_test, dtype=torch.float32),
                                  torch.tensor(y_test, dtype=torch.float32))
no_dupl_test_generator = torch.utils.data.DataLoader(no_dupl_test_set, **params)

### OR USE MEANDATA below
### Data loader for MeanData
# training_set = TensorDataset(torch.tensor(x_train_simple, dtype=torch.float32), torch.tensor(y_train_GNN, dtype=torch.float32))
# training_generator = torch.utils.data.DataLoader(training_set, **params)
# validation_set = TensorDataset(torch.tensor(x_dev_simple, dtype=torch.float32), torch.tensor(y_dev_GNN, dtype=torch.float32))
# validation_generator = torch.utils.data.DataLoader(validation_set, **params)
# no_dupl_test_set = TensorDataset(torch.tensor(x_test_simple, dtype=torch.float32),
#                                   torch.tensor(y_test_GNN, dtype=torch.float32))
# no_dupl_test_generator = torch.utils.data.DataLoader(no_dupl_test_set, **params)

In [50]:
x_train_simple[1].shape

(3, 31)

In [53]:
## Model parameters
dim_input = torch.tensor(x_train_GNN[1]).shape[1]   ## 31
dim_target = torch.tensor(y_train_GNN[1]).shape[0]  ## 12

## sage train with max pooling
dim_hidden = 256
dim_hidden_sage = 128  ## hidden nodes for GraphSage

batch_size = 32

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [54]:
train_with_soft_loss = False

if train_with_soft_loss:
    loss_function = SoftBCEWithLogitsLoss()
else:
    loss_function = nn.BCEWithLogitsLoss()

In [55]:
def load_checkpoint(model, optimizer, filename='checkpoint'):
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("=> loaded checkpoint '{}' (epoch {})".format(filename, checkpoint['epoch']))
    return model, optimizer

In [58]:
## Example run for sequential model. For training, need to run for more epochs

## Choose a classifier (LSTM, MLP, Bi-LSTM, RNN)

classifier = LSTM_model(dim_input, dim_target)
# classifier = Perceptron(dim_input, dim_target)
# classifier = BiLSTM(dim_input, dim_hidden, dim_target)
# classifier = RNN_model(dim_input, dim_target, dim_hidden)

optimizer = optim.RMSprop(classifier.parameters())
max_epochs, max_val_f1, patience = 5, 0, 2

print("Example run for sequential model. For training, need to run for more epochs.")


for epoch in range(max_epochs):
    f1s = []
    losses = []
    for inputs, labels in training_generator:
        classifier.zero_grad()

        inputs, labels = inputs.to(device), labels.to(device)
        output = classifier(inputs)
        if train_with_soft_loss:
            loss = loss_function(output, inputs, train_graph_GNN)
        else:
            loss = loss_function(output, labels)

        loss.backward()
        optimizer.step()
        losses.append(loss.data.mean().cpu())

        for out, lab in zip(output, labels):
            f1s.append(f1((out > 0.0).float(), lab).cpu())

    valid_loss, valid_f1 = validate_model(classifier, validation_generator, dev_graph_GNN, train_with_soft_loss, device)
    # noinspection PyStringFormat
    print('[%d/%d] Train Loss: %.3f, Train F1: %.3f, Val Loss: %.3f, Val F1: %.3f,' % (epoch + 1, 50, np.mean(losses), np.mean(f1s), valid_loss, valid_f1))

    # Early stopping
    if valid_f1 > max_val_f1:
        state = {'epoch': epoch + 1, 'state_dict': classifier.state_dict(), 'optimizer': optimizer.state_dict(), }
        torch.save(state, 'checkpoint')

        max_val_f1 = valid_f1
        max_epochs = 0
    else:
        max_epochs += 1
        if max_epochs >= patience:
            classifier, optimizer = load_checkpoint(classifier, optimizer)
            print("Stopped early and went back to Validation f1: %.3f" % max_val_f1)
            break

print("---------------------- DEVELOPMENT SET REPORT --------------------------")
evaluate_model(classifier, validation_generator, eval_graph_GNN, device)

print("--------------------- DEDUPLICATED TEST SET REPORT -------------------------")
evaluate_model(classifier, no_dupl_test_generator, eval_graph_GNN, device)


Example run for sequential model. For training, need to run for more epochs.
[1/50] Train Loss: 0.476, Train F1: 0.436, Val Loss: 0.401, Val F1: 0.468,
[2/50] Train Loss: 0.371, Train F1: 0.549, Val Loss: 0.394, Val F1: 0.569,
[3/50] Train Loss: 0.344, Train F1: 0.598, Val Loss: 0.361, Val F1: 0.555,
[4/50] Train Loss: 0.319, Train F1: 0.642, Val Loss: 0.362, Val F1: 0.562,
=> loaded checkpoint 'checkpoint' (epoch 2)
Stopped early and went back to Validation f1: 0.569
---------------------- DEVELOPMENT SET REPORT --------------------------
Hard F-Score (exact match): 0.569
Soft F-Score (best match): 0.806874
--------------------- DEDUPLICATED TEST SET REPORT -------------------------
Hard F-Score (exact match): 0.586
Soft F-Score (best match): 0.785284


In [59]:
## MLP
VHF = [0.650, 0.656, 0.657, 0.655, 0.649]
VSF = [0.915413, 0.909068, 0.876200, 0.901425, 0.888876]

THF = [0.702, 0.707, 0.702, 0.704 , 0.690]
TSF = [0.906208, 0.923331, 0.907678, 0.911401, 0.903732]

print(np.mean(VHF), np.mean(VSF), np.mean(THF), np.mean(TSF))

0.6534000000000001 0.8981964 0.7009999999999998 0.9104700000000001


In [60]:
## RNN
VHF = [0.299, 0.497, 0.441, 0.337, 0.357]
VSF = [0.426390, 0.755078, 0.689185, 0.578410, 0.599399]

THF = [0.244, 0.487, 0.424, 0.327, 0.359]
TSF = [0.359018, 0.684507,0.610320, 0.539102, 0.553551]
print(np.mean(VHF), np.mean(VSF), np.mean(THF), np.mean(TSF))

0.3862 0.6096923999999999 0.36819999999999997 0.5492996000000001


In [61]:
## Bi-LSTM
VHF = [0.640, 0.664, 0.651, 0.663, 0.652]
VSF = [0.925991, 0.935895, 0.914295, 0.881976, 0.936639]

THF = [0.671, 0.713, 0.685, 0.686, 0.699]
TSF = [0.907320, 0.931169, 0.904885, 0.892608, 0.935338]

print(np.mean(VHF), np.mean(VSF), np.mean(THF), np.mean(TSF))

0.6540000000000001 0.9189592000000001 0.6908 0.914264


In [62]:
## LSTM
VHF = [0.657, 0.656, 0.640, 0.667, 0.662]
VSF = [0.892167, 0.916457, 0.899351, 0.927221, 0.921875]

THF = [0.684, 0.697, 0.690, 0.722, 0.704]
TSF = [0.892062, 0.906511, 0.900117, 0.932719, 0.915623]
print(np.mean(VHF), np.mean(VSF), np.mean(THF), np.mean(TSF))

0.6564 0.9114142000000001 0.6994 0.9094064
